<a href="https://colab.research.google.com/github/kingamirasadi/Python-Pytorch-sklearn/blob/main/IMDB_NLP_Sklearn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [230]:
import pandas as pd
import numpy as np
import scipy
import nltk
from gensim.models import Word2Vec
from nltk.stem import PorterStemmer
import re   # text cleaning
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import GridSearchCV


In [231]:
data = pd.read_csv('/content/imdb_top_1000.csv')

In [232]:
data = data.drop('Poster_Link' , axis =1)

In [233]:
data.isnull().sum()

Series_Title       0
Released_Year      0
Certificate      101
Runtime            0
Genre              0
IMDB_Rating        0
Overview           0
Meta_score       157
Director           0
Star1              0
Star2              0
Star3              0
Star4              0
No_of_Votes        0
Gross            169
dtype: int64

In [234]:
# Step 2: Load the text data

In [235]:
genre_cert_df = data[['Genre', 'Certificate']]

In [236]:
genre_cert_df_filtered = genre_cert_df.dropna()

In [237]:
genre_cert_df_filtered.isnull().sum()

Genre          0
Certificate    0
dtype: int64

In [238]:
genre_cert_df_filtered.columns

Index(['Genre', 'Certificate'], dtype='object')

In [239]:
X = genre_cert_df_filtered['Genre']

In [240]:
y = genre_cert_df_filtered['Certificate']

In [241]:
X[1]

'Crime, Drama'

In [242]:
import re

def clean_text(text):
    text = re.sub(r'[^\w\s]', '' ,text.lower())  # Remove non-alphanumeric characters and lowercase
    return text


In [243]:
X = X.apply(clean_text)

In [244]:
# after cleaning text its time to convert text to vector

In [245]:
#we have many Options

In [246]:
# word2vec and TF-IDF are famous vectorizer

In [247]:
genre_list = X.tolist()
model = Word2Vec(genre_list, min_count=1, vector_size=100)  # Adjust parameters as needed


In [248]:
# after building Model it is time to convert text to vector

In [249]:
genre_vectors = []
for genre in X:
    try:
        genre_vector = model.wv[genre]
    except KeyError:
        genre_vector = np.zeros(100)  # Handle out-of-vocabulary words (optional)
    genre_vectors.append(genre_vector)


In [250]:
type(genre_vectors)

list

In [251]:
X_train, X_test, y_train, y_test = train_test_split(genre_vectors, y, test_size=0.2, random_state=42)


In [252]:
param_grid = {
    'max_depth': [3, 5, 8],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 4, 8],
    'criterion': ['gini', 'entropy']  # Decision tree splitting criteria
}

In [253]:
#tree =  DecisionTreeClassifier()

In [254]:
#tree.fit(X_train, y_train)


In [255]:
#y_pred = tree.predict(X_test)


In [256]:
grid_search = GridSearchCV(tree, param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


GridSearchCV(cv=5, estimator=DecisionTreeClassifier(),
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': [3, 5, 8], 'min_samples_leaf': [1, 4, 8],
                         'min_samples_split': [2, 5, 10]},
             scoring='accuracy')

In [257]:
# Get the best model and its parameters
best_model = grid_search.best_estimator_
best_params = grid_search.best_params_

In [258]:
print("Best Model Parameters:", best_params)

Best Model Parameters: {'criterion': 'gini', 'max_depth': 3, 'min_samples_leaf': 1, 'min_samples_split': 2}


In [259]:
y_pred = best_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Test Accuracy:", accuracy)

Test Accuracy: 0.31666666666666665
